# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
data_gr = data.groupby(['ProductName', 'CustomerID']).agg(sum_quantity = ('Quantity', 'sum')).sort_values(by='CustomerID')
data_gr

,,sum_quantity
ProductName,CustomerID,
Black Currants,33,1
"Pepper - White, Ground",33,1
"Pepper - Black, Whole",33,1
Pate - Cognac,33,1
"Beans - Kidney, Canned",33,1
...,...,...
Duck - Breast,98200,25
"Rum - Coconut, Malibu",98200,25
Wasabi Powder,98200,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
data_pv = data_gr.pivot_table(index='ProductName', columns='CustomerID', values='sum_quantity').fillna(0)
data_pv

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data_pv_t = data_pv.T
data_pv_t

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,50.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
data_dist = pdist(X=data_pv_t, metric='euclidean')
data_dist

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [9]:
data_sq = squareform(data_dist)
len(data_sq)

1000

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
data_eu = pd.DataFrame(data_sq,
                           index=data_pv.columns, 
                           columns=data_pv.columns)

data_eu

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


In [11]:
# Frame of reference

data_eu_norm = pd.DataFrame(1/(1 + squareform(pdist(data_pv_t, 'euclidean'))),
                                index=data_pv.columns,
                                columns=data_pv.columns)

data_eu_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [12]:
# Customer ID selected = 200

data_eu_norm_simil = data_eu_norm[200].sort_values(ascending=False)[1:]
#data_eu_norm_simil.head(5)
lst_more_similar_test = data_eu_norm_simil.head(5).index.tolist()
lst_more_similar_test

[3317, 1072, 3535, 1920, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
data_test = data.groupby(['ProductName', 'CustomerID'], as_index=False).agg(sum_quantity = ('Quantity', 'sum')).sort_values(by='CustomerID')
data_test

,ProductName,CustomerID,sum_quantity
7086,Black Currants,33,1
39284,"Pepper - White, Ground",33,1
38982,"Pepper - Black, Whole",33,1
38267,Pate - Cognac,33,1
3299,"Beans - Kidney, Canned",33,1
...,...,...,...
20080,Duck - Breast,98200,25
43502,"Rum - Coconut, Malibu",98200,25
57229,Wasabi Powder,98200,25
20640,Ecolab - Solid Fusion,98200,25


In [14]:
options = lst_more_similar_test
data_fil = data_test[data_test['CustomerID'].isin(options)]
data_fil

,ProductName,CustomerID,sum_quantity
10402,Cake - Box Window 10x10x2.5,1072,1
32651,"Mushroom - Porcini, Dry",1072,1
5671,Beef Wellington,1072,1
13095,Cheese - Parmesan Grated,1072,1
42150,Quiche Assorted,1072,1
...,...,...,...
38274,Pate - Cognac,3909,1
12379,Cheese - Cheddarsliced,3909,1
46016,Scampi Tail,3909,1
11833,"Cheese - Brie, Triple Creme",3909,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [15]:
data_fil_agg = data_fil.groupby('ProductName', as_index=False)['sum_quantity'].sum().sort_values(by='sum_quantity', ascending=False)
data_fil_agg.head(5)

,ProductName,sum_quantity
159,Soup - Campbells Bean Medley,4
103,Muffin - Carrot Individual Wrap,3
8,Bay Leaf,3
136,Pork - Kidney,3
129,"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [16]:
data_mer = data_fil_agg.merge(data_pv, left_on='ProductName', right_on='ProductName')
data_mer

,ProductName,sum_quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Soup - Campbells Bean Medley,4,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
1,Muffin - Carrot Individual Wrap,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0
2,Bay Leaf,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0
3,Pork - Kidney,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Pepper - Black, Whole",3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Kiwi,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203,Knife Plastic - White,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0
204,Lambcasing,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0
205,"Lentils - Red, Dry",1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
data_mer_fil = data_mer[data_mer[200]==0]
data_mer_fil[['ProductName', 'sum_quantity', 200]].head(5)

,ProductName,sum_quantity,200
0,Soup - Campbells Bean Medley,4,0.0
1,Muffin - Carrot Individual Wrap,3,0.0
2,Bay Leaf,3,0.0
3,Pork - Kidney,3,0.0
6,Wanton Wrap,3,0.0


In [18]:
lst_products_recom = data_mer_fil['ProductName'].head(5).tolist()
lst_products_recom

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [19]:
lst_customerid_unique = data['CustomerID'].unique().tolist()
lst_customerid_unique.sort()
#lst_customerid_unique

In [20]:
prod_recomm_dic ={}
prod_recomm_lst = []
for customer in lst_customerid_unique:
    data_eu_norm_simil = data_eu_norm[customer].sort_values(ascending=False)[1:]
    lst_most_similar = data_eu_norm_simil.head(5).index.tolist()
    data_fil = data_test[data_test['CustomerID'].isin(lst_most_similar)]
    data_fil_agg = data_fil.groupby('ProductName', as_index=False)['sum_quantity'].sum().sort_values(by='sum_quantity', ascending=False)
    data_mer = data_fil_agg.merge(data_pv, left_on='ProductName', right_on='ProductName')
    data_mer_fil = data_mer[data_mer[200]==0]
    lst_products_recom = data_mer_fil['ProductName'].head(5).tolist()
    prod_recomm_lst.append(lst_products_recom)
#print (prod_recomm_lst)
prod_recomm_dic = dict( zip(lst_customerid_unique,prod_recomm_lst))
prod_recomm_dic

{33: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Towels - Paper / Kraft',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein'],
 200: ['Soup - Campbells Bean Medley',
  'Muffin - Carrot Individual Wrap',
  'Bay Leaf',
  'Pork - Kidney',
  'Wanton Wrap'],
 264: ['Soupfoamcont12oz 112con',
  'Wine - Two Oceans Cabernet',
  'Bread - Italian Roll With Herbs',
  'Veal - Inside, Choice',
  'Potatoes - Idaho 100 Count'],
 356: ['Butter - Unsalted',
  'Veal - Inside, Choice',
  'Wine - Blue Nun Qualitatswein',
  'Table Cloth 81x81 White',
  'Veal - Sweetbread'],
 412: ['Sprouts - Baby Pea Tendrils',
  'Wine - Blue Nun Qualitatswein',
  'Veal - Inside, Choice',
  'Soup - Campbells Bean Medley',
  'Cheese - Cambozola'],
 464: ['Butter - Unsalted',
  'Cheese - Cambozola',
  'Veal - Inside, Choice',
  'Nut - Chestnuts, Whole',
  'Ketchup - Tomato'],
 477: ['Sprouts - Baby Pea Tendrils',
  'Wine - Blue Nun Qualitatswein',
  'Veal - Inside, Choice',
  'Soup - Campbell

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [21]:
df_final = pd.DataFrame.from_dict(prod_recomm_dic, orient='index')
df_final

,0,1,2,3,4
33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice",Potatoes - Idaho 100 Count
356,Butter - Unsalted,"Veal - Inside, Choice",Wine - Blue Nun Qualitatswein,Table Cloth 81x81 White,Veal - Sweetbread
412,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Veal - Inside, Choice",Soup - Campbells Bean Medley,Cheese - Cambozola
...,...,...,...,...,...
97928,Bouq All Italian - Primerba,Tea - Jasmin Green,"Soup - Campbells, Lentil",Arizona - Green Tea,"Cheese - Brie,danish"
98069,Skirt - 29 Foot,Chocolate - Dark,Beans - Kidney White,Milk - 1%,Longos - Grilled Salmon With Bbq
98159,Squid - Tubes / Tenticles 10/20,Chips Potato All Dressed - 43g,Table Cloth 81x81 White,Oil - Shortening - All - Purpose,Pernod
98185,Cod - Black Whole Fillet,Wine - Pinot Noir Latour,Chicken - Wieners,Crackers - Trio,Turkey - Oven Roast Breast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [22]:
data_cos_norm = pd.DataFrame(1/(1 + squareform(pdist(data_pv_t, 'cosine'))),
                                index=data_pv.columns,
                                columns=data_pv.columns)

In [23]:
prod_recomm_dic_2 ={}
prod_recomm_lst_2 = []
for customer in lst_customerid_unique:
    data_eu_norm_simil = data_cos_norm[customer].sort_values(ascending=False)[1:]
    lst_most_similar = data_eu_norm_simil.head(5).index.tolist()
    data_fil = data_test[data_test['CustomerID'].isin(lst_most_similar)]
    data_fil_agg = data_fil.groupby('ProductName', as_index=False)['sum_quantity'].sum().sort_values(by='sum_quantity', ascending=False)
    data_mer = data_fil_agg.merge(data_pv, left_on='ProductName', right_on='ProductName')
    data_mer_fil = data_mer[data_mer[200]==0]
    lst_products_recom = data_mer_fil['ProductName'].head(5).tolist()
    prod_recomm_lst_2.append(lst_products_recom)
#print (prod_recomm_lst)
prod_recomm_dic_2 = dict( zip(lst_customerid_unique,prod_recomm_lst_2))
prod_recomm_dic_2

{33: ['Knife Plastic - White',
  'Phyllo Dough',
  'Cake - Box Window 10x10x2.5',
  'Soup - Campbells, Beef Barley',
  'Onions - Cippolini'],
 200: ['Longos - Grilled Salmon With Bbq',
  'Snapple Lemon Tea',
  'General Purpose Trigger',
  'Thyme - Lemon, Fresh',
  'Tomatoes Tear Drop'],
 264: ['Pickerel - Fillets',
  'Muffin Mix - Blueberry',
  'Wine - Blue Nun Qualitatswein',
  'Water - Mineral, Natural',
  'Snapple - Iced Tea Peach'],
 356: ['Tomatoes Tear Drop',
  'Sobe - Tropical Energy',
  'Bread - English Muffin',
  'Thyme - Lemon, Fresh',
  'Corn Meal'],
 412: ['Banana - Leaves',
  'Salsify, Organic',
  'Beans - Wax',
  'Durian Fruit',
  'Whmis - Spray Bottle Trigger'],
 464: ['Lambcasing',
  'Wonton Wrappers',
  'Pail With Metal Handle 16l White',
  'Bread - Bistro White',
  'Sausage - Breakfast'],
 477: ['Halibut - Fletches',
  'Chocolate - Feathers',
  'Wine - Prosecco Valdobiaddene',
  'Pasta - Penne, Rigate, Dry',
  'Sunflower Seed Raw'],
 639: ['Juice - Apple Cider',
  'Se

In [24]:
df_final_cos = pd.DataFrame.from_dict(prod_recomm_dic_2, orient='index')
df_final_cos

,0,1,2,3,4
33,Knife Plastic - White,Phyllo Dough,Cake - Box Window 10x10x2.5,"Soup - Campbells, Beef Barley",Onions - Cippolini
200,Longos - Grilled Salmon With Bbq,Snapple Lemon Tea,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop
264,Pickerel - Fillets,Muffin Mix - Blueberry,Wine - Blue Nun Qualitatswein,"Water - Mineral, Natural",Snapple - Iced Tea Peach
356,Tomatoes Tear Drop,Sobe - Tropical Energy,Bread - English Muffin,"Thyme - Lemon, Fresh",Corn Meal
412,Banana - Leaves,"Salsify, Organic",Beans - Wax,Durian Fruit,Whmis - Spray Bottle Trigger
...,...,...,...,...,...
97928,Apricots - Halves,Cheese - Mozzarella,Tofu - Firm,Cookie Dough - Double,Blackberries
98069,Longos - Grilled Salmon With Bbq,Sprouts - Baby Pea Tendrils,Juice - Orange,Muffin Batt - Blueberry Passion,"Nut - Chestnuts, Whole"
98159,Squid - Tubes / Tenticles 10/20,Table Cloth 81x81 White,Yoghurt Tubes,"Liners - Banana, Paper","Water, Tap"
98185,Halibut - Fletches,Chicken - Wieners,Wasabi Powder,Cheese - Taleggio D.o.p.,"Cheese - Brie, Triple Creme"
